In [4]:
import numpy as np
import pandas as pd
import cv2

import matplotlib
#matplotlib.use('TkAgg')
#%matplotlib
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import scipy
import scipy.io as sio
from sklearn.metrics import mean_squared_error 
from scipy.spatial.distance import cdist
import random 

In [5]:
points_2d = np.loadtxt('../data/pts2d-norm-pic_a.txt')
points_3d = np.loadtxt('../data/pts3d-norm.txt')

print(points_2d)
print("")
print(points_3d)

[[ 1.0486 -0.3645]
 [-1.6851 -0.4004]
 [-0.9437 -0.42  ]
 [ 1.0682  0.0699]
 [ 0.6077 -0.0771]
 [ 1.2543 -0.6454]
 [-0.2709  0.8635]
 [-0.4571 -0.3645]
 [-0.7902  0.0307]
 [ 0.7318  0.6382]
 [-1.058   0.3312]
 [ 0.3464  0.3377]
 [ 0.3137  0.1189]
 [-0.431   0.0242]
 [-0.4799  0.292 ]
 [ 0.6109  0.083 ]
 [-0.4081  0.292 ]
 [-0.1109 -0.2992]
 [ 0.5129 -0.0575]
 [ 0.1406 -0.4527]]

[[ 1.5706 -0.149   0.2598]
 [-1.5282  0.9695  0.3802]
 [-0.6821  1.2856  0.4078]
 [ 0.4124 -1.0201 -0.0915]
 [ 1.2095  0.2812 -0.128 ]
 [ 0.8819 -0.8481  0.5255]
 [-0.9442 -1.1583 -0.3759]
 [ 0.0415  1.3445  0.324 ]
 [-0.7975  0.3017 -0.0826]
 [-0.4329 -1.4151 -0.2774]
 [-1.1475 -0.0772 -0.2667]
 [-0.5149 -1.1784 -0.1401]
 [ 0.1993 -0.2854 -0.2114]
 [-0.432   0.2143 -0.1053]
 [-0.7481 -0.384  -0.2408]
 [ 0.8078 -0.1196 -0.2631]
 [-0.7605 -0.5792 -0.1936]
 [ 0.3237  0.797   0.217 ]
 [ 1.3089  0.5786 -0.1887]
 [ 1.2323  1.4421  0.4506]]


In [9]:
def calculate_projection_matrix(points_2d, points_3d):
    """
    To solve for the projection matrix. You need to set up a system of
    equations using the corresponding 2D and 3D points:

                                                      [ M11      [ u1
                                                        M12        v1
                                                        M13        .
                                                        M14        .
    [ X1 Y1 Z1 1 0  0  0  0 -u1*X1 -u1*Y1 -u1*Z1        M21        .
      0  0  0  0 X1 Y1 Z1 1 -v1*X1 -v1*Y1 -v1*Z1        M22        .
      .  .  .  . .  .  .  .    .     .      .       *   M23   =    .
      Xn Yn Zn 1 0  0  0  0 -un*Xn -un*Yn -un*Zn        M24        .
      0  0  0  0 Xn Yn Zn 1 -vn*Xn -vn*Yn -vn*Zn ]      M31        .
                                                        M32        un
                                                        M33 ]      vn ]

    Then you can solve this using least squares with np.linalg.lstsq() or SVD.
    Notice you obtain 2 equations for each corresponding 2D and 3D point
    pair. To solve this, you need at least 6 point pairs.

    Args:
    -   points_2d: A numpy array of shape (N, 2)
    -   points_2d: A numpy array of shape (N, 3)

    Returns:
    -   M: A numpy array of shape (3, 4) representing the projection matrix
    """

    # Placeholder M matrix. It leads to a high residual. Your total residual
    # should be less than 1.
    M = np.asarray([[0.1768, 0.7018, 0.7948, 0.4613],
                    [0.6750, 0.3152, 0.1136, 0.0480],
                    [0.1020, 0.1725, 0.7244, 0.9932]])

    ###########################################################################
    # TODO: YOUR PROJECTION MATRIX CALCULATION CODE HERE
    ###########################################################################

    #raise NotImplementedError('`calculate_projection_matrix` function in ' +
        #'`student_code.py` needs to be implemented')
        
    A = np.zeros([len(points_2d)*2, 12])

    B = np.zeros([len(points_2d)*2, 1])

    k = 0
    for i in range(len(points_2d)):
        A[i + k][0], A[i + k][1], A[i + k][2], A[i + k][3] = points_3d[i][0], points_3d[i][1], points_3d[i][2], 1
        A[i + k][8], A[i + k][9], A[i + k][10], A[i + k][11] = points_2d[i][0]*points_3d[i][0], points_2d[i][0]*points_3d[i][1], points_2d[i][0]*points_3d[i][2], -points_2d[i][0]

        A[i + k + 1][4], A[i + k + 1][5], A[i + k + 1][6], A[i + k + 1][7] = points_3d[i][0], points_3d[i][1], points_3d[i][2], 1
        A[i + k + 1][8], A[i + k + 1][9], A[i + k + 1][10], A[i + k + 1][11] = points_2d[i][1]*points_3d[i][0], points_2d[i][1]*points_3d[i][1], points_2d[i][1]*points_3d[i][2], -points_2d[i][1]

        B[i + k] = points_2d[i][0]

        B[i + 1 + k] = points_2d[i][1]

        k = k + 1

    U, S, V = np.linalg.svd(A)
    #print(V)

    M = V[-1:12]
    M = np.reshape(M, (3,4))

    ###########################################################################
    # END OF YOUR CODE
    ###########################################################################

    return M

In [10]:
M = calculate_projection_matrix(points_2d, points_3d)

print('The projection matrix is\n', M)

The projection matrix is
 [[ 0.45827554 -0.29474237 -0.01395746  0.0040258 ]
 [-0.05085589 -0.0545847  -0.54105993 -0.05237592]
 [-0.10900958 -0.17834548  0.04426782  0.5968205 ]]
